In [1]:
import pandas as pd
from tabulate import tabulate
import os

In [2]:
files = os.listdir("./DATA/")
files.sort(reverse=True)
file = files[0] #me quedo con el último
csv_file = f'./DATA/{file}'  # Replace with the path to your CSV file
df = pd.read_csv(csv_file)

In [3]:
df.head()

,estado,tipooferta,jornada,ige,miercoles,martes,acargodireccion,cuilautor,supl_hasta,turno,...,reemp_motivo,descdistrito,jueves,nivelmodalidad,viernes,descripcionarea,vuelta,descripcioncargo,ult_movimiento,_version_
0,Publicada,DESIGNACIONES DOCENTES,JS,3115579,7:30 A 12:00,7:30 A 12:00,No,20275425894,9999-12-30T11:18:07.177Z,M,...,NaN,BERAZATEGUI,7:30 A 12:00,A,7:30 A 12:00,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-03-11T08:18:07.223Z,1826315164691988482
1,Publicada,DESIGNACIONES DOCENTES,JS,3116654,ALTERNADO,ALTERNADO,No,20147633158,9999-12-30T13:04:28.600Z,A,...,NaN,BERAZATEGUI,ALTERNADO,S,ALTERNADO,FONOAUDIOLOGO (/FO),NaN,FONOAUDIOLOGO (/FO),2025-03-11T10:04:29.833Z,1826315165031727108
2,Publicada,DESIGNACIONES DOCENTES,JS,3116245,9.30 A 10.30,NaN,No,20147633158,9999-12-30T12:30:21.887Z,M,...,NaN,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:30:23.140Z,1826315165165944841
3,Publicada,DESIGNACIONES DOCENTES,JS,3116363,NaN,NaN,No,20275425894,2026-02-28T00:00:00Z,M,...,Licencia no medica por ARTICULO 115(ART115),BERAZATEGUI,7:40 A 9:40,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:39:28.980Z,1826315165165944842
4,Publicada,DESIGNACIONES DOCENTES,JS,3116201,NaN,15:10 A 17:10,No,20147633158,9999-12-30T12:26:54.217Z,T,...,NaN,BERAZATEGUI,NaN,A,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:26:55.477Z,1826315165331619845


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 53 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   estado                 26 non-null     object 
 1   tipooferta             26 non-null     object 
 2   jornada                26 non-null     object 
 3   ige                    26 non-null     int64  
 4   miercoles              15 non-null     object 
 5   martes                 11 non-null     object 
 6   acargodireccion        26 non-null     object 
 7   cuilautor              26 non-null     int64  
 8   supl_hasta             26 non-null     object 
 9   turno                  26 non-null     object 
 10  idoferta               26 non-null     int64  
 11  domiciliodesempeno     26 non-null     object 
 12  cuilganador            0 non-null      float64
 13  sabado                 0 non-null      float64
 14  listadoorigenganador   0 non-null      float64
 15  id      

In [5]:
df

,estado,tipooferta,jornada,ige,miercoles,martes,acargodireccion,cuilautor,supl_hasta,turno,...,reemp_motivo,descdistrito,jueves,nivelmodalidad,viernes,descripcionarea,vuelta,descripcioncargo,ult_movimiento,_version_
0,Publicada,DESIGNACIONES DOCENTES,JS,3115579,7:30 A 12:00,7:30 A 12:00,No,20275425894,9999-12-30T11:18:07.177Z,M,...,NaN,BERAZATEGUI,7:30 A 12:00,A,7:30 A 12:00,PRECEPTOR (/PR),NaN,PRECEPTOR (/PR),2025-03-11T08:18:07.223Z,1826315164691988482
1,Publicada,DESIGNACIONES DOCENTES,JS,3116654,ALTERNADO,ALTERNADO,No,20147633158,9999-12-30T13:04:28.600Z,A,...,NaN,BERAZATEGUI,ALTERNADO,S,ALTERNADO,FONOAUDIOLOGO (/FO),NaN,FONOAUDIOLOGO (/FO),2025-03-11T10:04:29.833Z,1826315165031727108
2,Publicada,DESIGNACIONES DOCENTES,JS,3116245,9.30 A 10.30,NaN,No,20147633158,9999-12-30T12:30:21.887Z,M,...,NaN,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:30:23.140Z,1826315165165944841
3,Publicada,DESIGNACIONES DOCENTES,JS,3116363,NaN,NaN,No,20275425894,2026-02-28T00:00:00Z,M,...,Licencia no medica por ARTICULO 115(ART115),BERAZATEGUI,7:40 A 9:40,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:39:28.980Z,1826315165165944842
4,Publicada,DESIGNACIONES DOCENTES,JS,3116201,NaN,15:10 A 17:10,No,20147633158,9999-12-30T12:26:54.217Z,T,...,NaN,BERAZATEGUI,NaN,A,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:26:55.477Z,1826315165331619845
5,Publicada,DESIGNACIONES DOCENTES,JS,3116300,16:55 A 17:55,NaN,No,20147633158,2025-04-11T00:00:00Z,T,...,Licencia no medica por ARTICULO 114(ART114NOme...,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:34:37.213Z,1826315165334765574
6,Publicada,DESIGNACIONES DOCENTES,JS,3116619,8 a 11,8 a 11,No,20147633158,9999-12-30T13:01:41.657Z,M,...,NaN,BERAZATEGUI,8 a 11,L,8 a 11,MAESTRO DE CICLO (/MC),NaN,MAESTRO DE CICLO (/MC),2025-03-11T10:01:42.907Z,1826315165334765579
7,Publicada,DESIGNACIONES DOCENTES,JS,3116556,NaN,11:40-12:40,No,20147633158,2026-02-28T00:00:00Z,M,...,Relevo por servicios provisorios orden tecnico...,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T09:56:30.890Z,1826315165337911297
8,Publicada,DESIGNACIONES DOCENTES,JS,3117114,NaN,21:25 A 22:15,No,20275425894,9999-12-30T13:43:48.367Z,V,...,NaN,BERAZATEGUI,NaN,E,NaN,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T10:43:46.290Z,1826315165343154184
9,Publicada,DESIGNACIONES DOCENTES,JS,3117150,NaN,NaN,No,20275425894,9999-12-30T13:43:48.367Z,V,...,NaN,BERAZATEGUI,NaN,E,19:40 A 21:25,CONSTRUCCION DE CIUDADANIA (CCD),NaN,CONSTRUCCION DE CIUDADANIA (CCD),2025-03-11T10:46:08.783Z,1826315165386145797


In [35]:
orden_distritos = {'QUILMES':2,'AVELLANEDA':1,'BERAZATEGUI':0}
ofertas_por_distrito = df.groupby(['descdistrito','descripcioncargo'])['ige'].nunique().reset_index()
ofertas_por_distrito.columns = ['Distrito','Cargo','Cantidad']
ofertas_por_distrito["order"] = ofertas_por_distrito.Distrito.map(orden_distritos)
ofertas_por_distrito = ofertas_por_distrito.sort_values(by=['order','Cantidad'],ascending=False)
ofertas_por_distrito.drop(columns="order",inplace=True)
ofertas_por_distrito = tabulate(
    ofertas_por_distrito, 
    headers=ofertas_por_distrito.columns, 
    tablefmt="HTML",
    showindex=False,
)
print(ofertas_por_distrito)


Distrito     Cargo                               Cantidad
-----------  --------------------------------  ----------
QUILMES      CONSTRUCCION DE CIUDADANIA (CCD)           2
QUILMES      PRECEPTOR (/PR)                            1
AVELLANEDA   CONSTRUCCION DE CIUDADANIA (CCD)           3
AVELLANEDA   PRECEPTOR (/PR)                            3
BERAZATEGUI  CONSTRUCCION DE CIUDADANIA (CCD)          13
BERAZATEGUI  PRECEPTOR (/PR)                            2
BERAZATEGUI  FONOAUDIOLOGO (/FO)                        1
BERAZATEGUI  MAESTRO DE CICLO (/MC)                     1


In [37]:
ofertas_por_distrito

'Distrito     Cargo                               Cantidad\n-----------  --------------------------------  ----------\nQUILMES      CONSTRUCCION DE CIUDADANIA (CCD)           2\nQUILMES      PRECEPTOR (/PR)                            1\nAVELLANEDA   CONSTRUCCION DE CIUDADANIA (CCD)           3\nAVELLANEDA   PRECEPTOR (/PR)                            3\nBERAZATEGUI  CONSTRUCCION DE CIUDADANIA (CCD)          13\nBERAZATEGUI  PRECEPTOR (/PR)                            2\nBERAZATEGUI  FONOAUDIOLOGO (/FO)                        1\nBERAZATEGUI  MAESTRO DE CICLO (/MC)                     1'

In [40]:
ofertas_por_escuela = df.groupby(['descdistrito','escuela','domiciliodesempeno','descripcioncargo'])['ige'].nunique().reset_index()
ofertas_por_escuela.columns = ['Distrito','Escuela','Dirección','Cargo','Cantidad']
ofertas_por_escuela["order"] = ofertas_por_escuela.Distrito.map(orden_distritos)
ofertas_por_escuela = ofertas_por_escuela.sort_values(by=['order','Cantidad'],ascending=False)
ofertas_por_escuela.drop(columns="order",inplace=True)
ofertas_por_escuela = tabulate(
    ofertas_por_escuela, 
    headers=ofertas_por_escuela.columns, 
    tablefmt="html",
    showindex=False,
)
ofertas_por_escuela

Distrito,Escuela,Dirección,Cargo,Cantidad
QUILMES,0085MS0002,844,CONSTRUCCION DE CIUDADANIA (CCD),1
QUILMES,0085MS0006,ALEM,PRECEPTOR (/PR),1
QUILMES,0085MS0037,CALLE 835,CONSTRUCCION DE CIUDADANIA (CCD),1
AVELLANEDA,0005MS0023,DE LA SERNA,CONSTRUCCION DE CIUDADANIA (CCD),2
AVELLANEDA,0005MS0001,ELIZALDE,CONSTRUCCION DE CIUDADANIA (CCD),1
AVELLANEDA,0005MS0003,DEHEZA,PRECEPTOR (/PR),1
AVELLANEDA,0005MS0006,GIRIBONE,PRECEPTOR (/PR),1
AVELLANEDA,0005MS0013,BERUTI,PRECEPTOR (/PR),1
BERAZATEGUI,0119MS0005,148,CONSTRUCCION DE CIUDADANIA (CCD),2
BERAZATEGUI,0119MS0018,27 E/ 106 Y 107,CONSTRUCCION DE CIUDADANIA (CCD),2


In [8]:
cantidad_ofertas = df["ige"].nunique()
cantidad_ofertas

26